### Image Props Again on Boundary Image

In [ ]:
# Produce binary image with cells=1 and background=0 and gives each cell a unique identifier
im_labeled, n_labels = skimage.measure.label(im_mark_boundaries, background=0, return_num=True)

# Extract the cells from background
im_props = skimage.measure.regionprops(im_labeled)

In [ ]:
# These are better determined by the whole dataset
min_area = 2000
max_eccentricity = .8

# Loop through image properties and delete small objects and objects that aren't circular enough
n_regions = 0
for prop in im_props:
    # Measure of roundness, ensuring cells are at least somewhat round
    area = prop.area
    if prop.area < min_area:
        im_mark_boundaries[im_labeled == prop.label] = False
    else:
        n_regions += 1
'''elif prop.eccentricity > max_eccentricity:
im_mark_boundaries[im_labeled == prop.label] = False'''

In [ ]:
bokeh.io.show(bi1x.viz.imshow(im_mark_boundaries))

### Image Props on Boundary Image With Dimensional Separation

In [ ]:
array = np.array([find_boundaries[200], find_boundaries[201]])

In [ ]:
bokeh.io.show(bi1x.viz.imshow(array))

In [ ]:
pixel_array = np.empty(shape=(512, 512), dtype=np.byte)

for x in range(len(find_boundaries)):
    for y in range(len(find_boundaries)):
        pixel = find_boundaries[x][y]
        pixel_array[x][y] = pixel

In [ ]:
bokeh.io.show(bi1x.viz.imshow(pixel_array))

In [ ]:
# Median filtering
im_med_filt = skimage.filters.median(im_dct['Data/out/max_c3_test_im.tiff'])

# Otsu threshold
otsu_thresh = skimage.filters.threshold_otsu(im_med_filt)

im_otsu_thresh = im_med_filt > otsu_thresh

# Clear partial cells around edges
im_cleared_edges = skimage.segmentation.clear_border(im_otsu_thresh, buffer_size=10)

# Watershed segmentation, try to split the cells from each other
distance = ndimage.distance_transform_edt(im_med_filt)
local_maxi = skimage.feature.peak_local_max(distance, indices=False, footprint=np.ones((3, 3)), labels=image)
markers = skimage.morphology.label(local_maxi)
labels_ws = skimage.morphology.watershed(-distance, markers, mask=image)

# Random Walker Segmentation
im_to_invert = im_med_filt.astype(np.int)
markers[~im_to_invert] = -1
labels_rw = skimage.segmentation.random_walker(im_to_invert, markers)


# Random Props Segmentation
# Produce binary image with cells=1 and background=0 and gives each cell a unique identifier
im_labeled, n_labels = skimage.measure.label(im_cleared_edges, background=0, return_num=True)
# Extract the cells from background
im_props = skimage.measure.regionprops(im_labeled)
# These are better determined by the whole dataset
min_area = 5000
max_eccentricity = .7
# Loop through image properties and delete small objects and objects that aren't circular enough
n_regions = 0
for prop in im_props:
    # Measure of roundness, ensuring cells are at least somewhat round
    area = prop.area
    if prop.area < min_area:
        im_cleared_edges[im_labeled == prop.label] = False
    else:
        n_regions += 1
'''    elif prop.eccentricity > max_eccentricity:
        im_cleared_edges[im_labeled == prop.label] = False'''


# Chan Vese
# Changing the mu value changes how circular of objects it's sensing
im_chan_vese = skimage.segmentation.chan_vese(im_med_filt, mu=.75)
segmentation = im_chan_vese[0]
final_level_set = im_chan_vese[1]
energies = im_chan_vese[2]


# Feltzenszwalb Segmentation
im_felzenswalb = skimage.segmentation.felzenszwalb(im_med_filt, scale=2.0, multichannel=False)


# Find Boundaries
find_boundaries = skimage.segmentation.find_boundaries(im_cleared_edges, connectivity=10, mode='inner')

# Mark Boundaries
im_mark_boundaries = skimage.segmentation.mark_boundaries(im_cleared_edges, find_boundaries)

In [ ]:
def combine_xist_dapi1(im_xist, im_dapi, min_area):
    '''Takes an input of an image array as a numpy array segmented in the 
    DAPI channel and a numpy array segmented in the XIST channel and removes
    XIST signal not within cells. Also compresses the DAPI channel into 2D 
    and merges the DAPI and XIST images into different layers of an RGB image.
    Must meet the minimum area (between 0 and 1)
    
    Returns a numpy array of cells with XIST signal in them.'''
    labeled_im_xist = skimage.measure.label(im_xist)
    labeled_im_dapi = skimage.measure.label(im_dapi)
    
    # Making XIST im 2D
    for region in skimage.measure.regionprops(labeled_im_xist):
        for coord in region.coords:
            (x,y,z) = coord
            labeled_im_xist[x][y][0] = 0
            labeled_im_xist[x][y][1] = 1
            labeled_im_xist[x][y][2] = 0
            
    # Initialize dictionary for area counts
    xist_area_count_dct = {}
    dapi_area_count_dct = {}
    
            
    # Make DAPI 2D and add Xist onto layer of RGB image
    for region in skimage.measure.regionprops(labeled_im_dapi):
        # Initialize area counts for XIST in a cell, restarts with each new cell
        xist_area_count = 0
        dapi_area_count = 0
        for coord in region.coords:
            (x, y, z) = coord
            labeled_im_dapi[x][y][0] = 1
            labeled_im_dapi[x][y][2] = 0
            dapi_area_count += 1
            if labeled_im_xist[x][y][1] == 1:
                xist_area_count += 1
        # Need to raise error if min_area not between 0 and 1
        print(xist_area_count / dapi_area_count)
        
        # If the ratio of xist to cell size is greater than the min area, add XIST to image
        if xist_area_count / dapi_area_count > min_area:
            for coord in region.coords:
                if labeled_im_xist[x][y][1] == 1:
                    labeled_im_dapi[x][y][1] = 1
                    
                # Otherwise, this cell won't be used for data, so remove it from the image
                else:
                    labeled_im_dapi[x][y][0] = 0
                    labeled_im_dapi[x][y][1] = 0
                    labeled_im_dapi[x][y][2] = 0
        xist_area_count_dct[region.centroid] = xist_area_count
        dapi_area_count_dct[region.centroid] = dapi_area_count
    return (labeled_im_dapi, xist_area_count_dct, dapi_area_count_dct)

In [ ]:
def combine_xist_dapi2(im_xist, im_dapi):
    '''Takes an input of an image array as a numpy array segmented in the 
    DAPI channel and a numpy array segmented in the XIST channel and removes
    XIST signal not within cells. Also compresses the DAPI channel into 2D 
    and merges the DAPI and XIST images into different layers of an RGB image.
    
    Returns a numpy array of cells with XIST signal in them.'''
    labeled_im_xist = skimage.measure.label(im_xist)
    labeled_im_dapi = skimage.measure.label(im_dapi)
    
    # Making XIST im 2D
    for region in skimage.measure.regionprops(labeled_im_xist):
        for coord in region.coords:
            (x,y,z) = coord
            labeled_im_xist[x][y][0] = 0
            labeled_im_xist[x][y][1] = 1
            labeled_im_xist[x][y][2] = 0
            
    # Initialize dictionary for area counts
    xist_area_count_dct = {}
    
            
    # Make DAPI 2D and add Xist onto layer of RGB image
    for region in skimage.measure.regionprops(labeled_im_dapi):
        # Initialize area counts for XIST in a cell, restarts with each new cell
        xist_area_count = 0
        for coord in region.coords:
            (x, y, z) = coord
            labeled_im_dapi[x][y][0] = 1
            labeled_im_dapi[x][y][2] = 0
            if labeled_im_xist[x][y][1] == 1:
                labeled_im_dapi[x][y][1] = 1
                xist_area_count += 1
        xist_area_count_dct[region.centroid] = xist_area_count
    return (labeled_im_dapi, xist_area_count_dct)